# How to extract 3D structure of the nebulae from the LVM data simulator

1. Initialize the LVM Field

In [2]:
import os
from lvmdatasimulator.field import LVMField
import astropy.units as u
import astropy.table as Table
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
#os.chdir('/Users/mors/Science/LVM/Simulator_example/')  # Your top-level working directory
os.chdir('/home/amrita/LVM/lvmnebular/')

In [3]:
#Some random of the LVM Field
'''
ra = 12.34 # u.degree
dec = -20.35 # u.degree
fov_size = 35 # u.arcmin
fov_pixel = 1 # u.arcsec
distance = 50 # * u.kpc
sys_vel = 0 #* u.km / u.s
turbulent_sigma = 20 #* u.km / u.s
preserve_kinematics = False
'''
#specifing parameters
ra = 90.5625 # u.degree
dec = 4.998333 # u.degree
fov_size = 10 # u.arcmin
fov_pixel = 10 # u.arcsec
distance = 16.0 * u.kpc
sys_vel = 20 #* u.km / u.s
turbulent_sigma = 15 #* u.km / u.s
preserve_kinematics = False

unit_ra = u.degree
unit_dec = u.degree
unit_size = u.arcmin
unit_pixel = u.arcsec

name = 'LVM_extract3D'
my_lvmfield = LVMField(ra=ra, dec=dec, size=fov_size, pxsize=fov_pixel, name=name,
                       unit_ra=unit_ra, unit_dec=unit_dec, unit_size=unit_size, unit_pxsize=unit_pixel,
                       ism_params={'distance': distance, 'sys_velocity': sys_vel, 'turbulent_sigma': turbulent_sigma,
                                   'preserve_kinematics': preserve_kinematics})

2. Define the properties of the nebula to be simulated
Let's simulate only a Bubble in our FOV

In [4]:
'''
my_nebulae = [{'type': 'Bubble3D',
               'max_brightness': 1.e-14,
               'thickness': 0.85,
               'radius': 10*u.pc,
               'expansion_velocity': 15,
                'turbulent_sigma': 15,
                'sys_velocity': 0,
               'model_params': {'Z': 0.6, 'qH': 50, 'nH': 100, 'Geometry': 'Cloud', 'Teff': 35000.},
               'model_type': 'cloudy',
               'offset_RA':0,
               'offset_DEC':-20},
              ]
'''
my_nebulae = [{'type': 'Bubble3D',
               'max_brightness': 5e-14,
               'thickness': 0.8,
               'radius': 18*u.pc,
               'expansion_velocity': 10,
                'turbulent_sigma': 15,
                'sys_velocity': 10,
               'model_params': {'Z': 1.0, 'qH': 50., 'nH': 100, 'Geometry': 'Shell', 'Teff': 40000.},
               'model_type': 'cloudy',
               'offset_RA':0,
               'offset_DEC':0},
              ]

3. Add this nebula to the LVM Field

In [5]:
my_lvmfield.add_nebulae(my_nebulae)

[INFO]: Start generating 1 nebulae
[WARNING]: Use the closest pre-computed model with id = Shell_921. Its parameters: Geometry = Shell; Z = 1.0; qH = 50.0; LogLsun = 6.5; Teff = 30000.0; nH = 100.0; 


4. Perfect! Now you have access to all the properties of the nebulae considered in the simulator.

In [6]:
print(my_lvmfield.ism.nebulae_objects[0].max_brightness)
print(my_lvmfield.ism.nebulae_objects[0].thickness)
print(my_lvmfield.ism.nebulae_objects[0].radius)
print(my_lvmfield.ism.nebulae_objects[0].expansion_velocity)


5e-14 erg / (s arcsec2 cm2)
0.8
18.0 pc
10.0 km / s


One access the relative 3D brightness distribution in cartesian or in spherical coordinates.

In [7]:
print(my_lvmfield.ism.nebulae_objects[0]._brightness_3d_cartesian.shape)  # nz, ny, nx, where nx is along line-of-sight
print(my_lvmfield.ism.nebulae_objects[0]._brightness_3d_spherical.shape)  # rad, theta, phi

c3D=my_lvmfield.ism.nebulae_objects[0]._brightness_3d_cartesian #
print(c3D[10,20,30]) #considering uniform brightness in each shell=1 (default)


(49, 49, 100)
(70, 90, 90)
2.204585537918871e-06


In [8]:
print(my_lvmfield.ism.nebulae_objects[0]._cartesian_x_grid.shape)  # this one is along line-of-sight
print(my_lvmfield.ism.nebulae_objects[0]._cartesian_y_grid.shape)
print(my_lvmfield.ism.nebulae_objects[0]._cartesian_z_grid.shape)

(100,)
(49,)
(49,)


In [9]:
print(my_lvmfield.ism.nebulae_objects[0]._theta_grid,
      my_lvmfield.ism.nebulae_objects[0]._rad_grid,
      my_lvmfield.ism.nebulae_objects[0]._phi_grid)

[0.         0.03529879 0.07059759 0.10589638 0.14119518 0.17649397
 0.21179276 0.24709156 0.28239035 0.31768914 0.35298794 0.38828673
 0.42358553 0.45888432 0.49418311 0.52948191 0.5647807  0.6000795
 0.63537829 0.67067708 0.70597588 0.74127467 0.77657346 0.81187226
 0.84717105 0.88246985 0.91776864 0.95306743 0.98836623 1.02366502
 1.05896382 1.09426261 1.1295614  1.1648602  1.20015899 1.23545779
 1.27075658 1.30605537 1.34135417 1.37665296 1.41195175 1.44725055
 1.48254934 1.51784814 1.55314693 1.58844572 1.62374452 1.65904331
 1.69434211 1.7296409  1.76493969 1.80023849 1.83553728 1.87083607
 1.90613487 1.94143366 1.97673246 2.01203125 2.04733004 2.08262884
 2.11792763 2.15322643 2.18852522 2.22382401 2.25912281 2.2944216
 2.32972039 2.36501919 2.40031798 2.43561678 2.47091557 2.50621436
 2.54151316 2.57681195 2.61211075 2.64740954 2.68270833 2.71800713
 2.75330592 2.78860471 2.82390351 2.8592023  2.8945011  2.92979989
 2.96509868 3.00039748 3.03569627 3.07099507 3.10629386 3.141592

Similar for 4D, where first dimension is a line ID

In [10]:
print(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_cartesian.shape)
print(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_spherical.shape)

#print(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_cartesian)
#print(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_spherical)

#ind5007=100
#my_lvmfield.ism.nebulae_objects[0]._brightness_4d_cartesian[ind5007,:,:,:]=pertemcube

(141, 49, 49, 100)
(141, 70, 90, 90)


In [11]:
hdu1=fits.PrimaryHDU(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_cartesian)
hdul1=fits.HDUList([hdu1])
hdul1.writeto('cube_4D_cart.fits', overwrite=True)

In [12]:
hdu1=fits.PrimaryHDU(my_lvmfield.ism.nebulae_objects[0]._brightness_3d_cartesian)
hdul1=fits.HDUList([hdu1])
hdul1.writeto('cube_3D_cart.fits', overwrite=True)

In [13]:
hdu=fits.PrimaryHDU(my_lvmfield.ism.nebulae_objects[0]._brightness_4d_spherical)
hdul=fits.HDUList([hdu])
hdul.writeto('cube_sph.fits', overwrite=True) #storing 4D cube in fits format (spherical coordinate)

In [53]:
#Reading fits file
hdul1=fits.open('cube_sph.fits')
data=hdul1[0].data
print(data[100, 10, 20, 30], data[10, 20, 10, 10], data[140, 20, 30, 10], data[140, 69, 89, 89], data.shape)


0.0 1.6485494914890712e-08 4.619502831194954e-08 1.8416174297160965e-10 (141, 70, 90, 90)


In [63]:
with fits.open('cube_4D_cart.fits') as hdul:
    data=hdul[0].data

print(data[:, 10,20,30], data.shape, data[0, :].shape)

[6.90848741e-11 1.47518404e-11 5.58428223e-08 9.84310992e-09
 8.94512595e-09 9.14154304e-09 9.79337848e-09 1.07724139e-08
 1.20666943e-08 1.37171966e-08 1.58000536e-08 1.84219449e-08
 1.91813305e-11 2.17523536e-08 3.68165132e-10 2.60326513e-08
 1.64081016e-07 2.18903400e-07 3.16173544e-08 3.90474520e-08
 4.91519086e-08 4.26945953e-08 7.65922524e-12 2.38747426e-12
 5.84546125e-08 5.06772576e-15 2.81338566e-12 5.54975489e-11
 8.34737334e-08 3.38947721e-12 4.26014852e-10 1.83674938e-10
 1.53812407e-15 1.25855215e-07 4.42996323e-12 1.43533263e-11
 4.04876789e-09 1.30966428e-09 2.04311386e-07 5.03397205e-13
 7.63162377e-12 5.68908408e-10 1.01513383e-09 3.69259287e-07
 7.43323785e-10 1.37979364e-17 1.47029799e-11 5.21834870e-10
 8.70629904e-10 3.18230959e-11 2.54288773e-10 9.69618170e-11
 1.18708063e-09 2.17948394e-08 4.87515077e-10 4.88400461e-21
 6.92393858e-09 0.00000000e+00 1.53474769e-12 1.74448735e-09
 0.00000000e+00 3.98576682e-09 2.32062209e-09 8.28977273e-10
 8.71068496e-10 7.903519

In [92]:
print(data.shape, np.max(data[:, 10, 20, 30]), data[:, 15, 25, 35], data[:, 0, 0, 0].shape)

(141, 49, 49, 100) 2.258243067783357e-06 [3.33870847e-09 6.28475391e-09 5.67530107e-08 1.00823341e-08
 9.19968952e-09 9.42127632e-09 1.01053033e-08 1.11228709e-08
 1.24630785e-08 1.41662218e-08 1.63162063e-08 1.90200017e-08
 8.99213086e-09 2.24550440e-08 6.39659701e-10 2.68732182e-08
 2.23174875e-07 2.95972343e-07 3.26463923e-08 4.03321732e-08
 5.07903260e-08 4.46510015e-08 7.45456418e-09 7.93162083e-10
 6.10764072e-08 2.52565009e-10 1.05159076e-09 6.27986809e-08
 8.71391732e-08 1.44401250e-09 5.83776304e-10 6.69199031e-09
 7.66576973e-11 1.31304689e-07 2.07672904e-09 1.34435995e-08
 4.16625318e-09 1.34696934e-09 2.13079793e-07 8.62949750e-10
 1.99558957e-09 3.68334619e-10 6.28692960e-10 3.84917182e-07
 4.60363075e-10 1.31953193e-11 3.61072353e-09 3.23175085e-10
 5.41987713e-10 2.80173558e-08 2.43127156e-10 9.25275856e-11
 1.76907295e-09 3.23839131e-08 7.28924454e-10 3.30334595e-14
 1.03186462e-08 8.11321212e-13 2.35501752e-09 2.60839997e-09
 5.65427694e-13 5.92208753e-09 3.45834949e-0